<a href="https://colab.research.google.com/github/messiah2305/Catboost/blob/main/vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
train_path="/content/drive/MyDrive/Tealeaf_finaldataset/Train"
test_path="/content/drive/MyDrive/Tealeaf_finaldataset/Test"
val_path="/content/drive/MyDrive/Tealeaf_finaldataset/Validation"

In [4]:
x_train=[]

for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_train.append(img_arr)

x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_test.append(img_arr)

x_val=[]

for folder in os.listdir(val_path):

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_val.append(img_arr)

In [5]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)

In [6]:
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

In [9]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 6,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 6,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 6,
                                            class_mode = 'sparse')

Found 709 images belonging to 8 classes.
Found 88 images belonging to 8 classes.
Found 89 images belonging to 8 classes.


In [10]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes

In [11]:
training_set.class_indices

{'Anthracnose': 0,
 'algal leaf': 1,
 'bird eye spot': 2,
 'brown blight': 3,
 'gray light': 4,
 'healthy': 5,
 'red leaf spot': 6,
 'white spot': 7}

In [12]:
train_y.shape,test_y.shape,val_y.shape

((709,), (88,), (89,))

In [13]:
IMAGE_SIZE = [224, 224]

In [14]:
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80150528/80134624 [==============================] - 0s 0us/step


In [15]:
for layer in vgg.layers:
    layer.trainable = False

In [16]:
x = Flatten()(vgg.output)
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction = Dense(8, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

In [19]:
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=10,
  callbacks=[early_stop],
  batch_size=6,shuffle=True)

Epoch 1/10
119/119 [==============================] - 20s 72ms/step - loss: 2.3537 - accuracy: 0.3399 - val_loss: 5.4297 - val_accuracy: 0.0899
Epoch 2/10
119/119 [==============================] - 7s 58ms/step - loss: 1.1393 - accuracy: 0.6220 - val_loss: 6.2847 - val_accuracy: 0.1348
Epoch 3/10
119/119 [==============================] - 7s 58ms/step - loss: 0.7206 - accuracy: 0.7616 - val_loss: 6.1667 - val_accuracy: 0.0899
Epoch 4/10
119/119 [==============================] - 7s 59ms/step - loss: 0.4005 - accuracy: 0.8604 - val_loss: 6.6277 - val_accuracy: 0.1236
Epoch 5/10
119/119 [==============================] - 7s 59ms/step - loss: 0.2782 - accuracy: 0.9069 - val_loss: 6.5093 - val_accuracy: 0.0337
Epoch 6/10
119/119 [==============================] - 7s 59ms/step - loss: 0.2284 - accuracy: 0.9394 - val_loss: 6.6824 - val_accuracy: 0.0449
Epoch 6: early stopping
